In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from pathlib import Path

### Interest Rates

In [ ]:
# load historical data on Euribor rates
euribor_3m = pd.read_csv(Path("./data/euribor_3m_data.csv")).pipe(
    lambda x: x.assign(Date = pd.to_datetime(x["Date"], format="%Y%b"))
)
euribor_6m = pd.read_csv(Path("./data/euribor_6m_data.csv")).pipe(
    lambda x: x.assign(Date = pd.to_datetime(x["Date"], format="%Y%b"))
)
euribor_12m = pd.read_csv(Path("./data/euribor_12m_data.csv")).pipe(
    lambda x: x.assign(Date = pd.to_datetime(x["Date"], format="%Y%b"))
)

# join the dataframes together, drop unnecessary columns,
# select only complete data points and calculate spreads and relative change
# sort the values based on date to get moving average
# calculation correct
all_euribors = (
    euribor_3m
    .merge(euribor_6m, how="left", on="Date")
    .merge(euribor_12m, how="left", on="Date")
    .pipe(
        lambda x: x.drop(x.filter(regex="Status|Comment").columns, axis=1)
    )
    .dropna(axis=0, how="any")
    .assign(
        Spread3M = lambda x: x["Euribor3M"] - x["Euribor12M"],
        Spread6M = lambda x: x["Euribor6M"] - x["Euribor12M"]
    )
    .pipe(
        lambda x: x.join((
            x[x.columns[x.columns.str.startswith("Euribor")]]
            .pct_change()
            .apply(lambda y: 100*y, axis=1)
            .rename(lambda y: "".join(["RelChange", re.findall("\d+", y)[0], "M"]), axis=1)
        ))
    )
    .sort_values("Date", axis=0)
)

all_euribors.head()

In [ ]:
plt.figure()
all_euribors.plot(x="Date", y=["Euribor3M", "Euribor6M", "Euribor12M"], grid=True)
plt.axis("tight")
plt.show()

### Spreads

In [ ]:
def calculate_ma(df, window_length):
    """"
        Function to calculate moving average of all columns given the window length.

        Parameters:
            df: a pandas DataFrame representing the data
            window_length: number of periods included in a single time window
    """

    res = (
        df
        .drop("Date", axis=1)
        .rolling(window_length)
        .mean()
        .assign(Date = df["Date"])
    )

    return res

# calculate moving averages of interest rates and spreads
window_6m = calculate_ma(all_euribors, 6)
window_1y = calculate_ma(all_euribors, 12)
window_2y = calculate_ma(all_euribors, 24)
window_10y = calculate_ma(all_euribors, 10*12)

In [ ]:
plt.figure()
window_10y.plot(x="Date", y=["Spread3M", "Spread6M"],
                title="Moving average, 10Y", grid=True)
plt.axis("tight")
plt.show()

plt.figure()
window_2y.plot(x="Date", y=["Spread3M", "Spread6M"],
                title="Moving average, 2Y", grid=True)
plt.axis("tight")
plt.show()

plt.figure()
window_1y.plot(x="Date", y=["Spread3M", "Spread6M"],
                title="Moving average, 1Y", grid=True)
plt.axhline(y=-0.1, color="g", linestyle="--", label="10bp")
plt.axis("tight")
plt.legend()
plt.show()

In [ ]:
plt.figure()
all_euribors.plot(x="Date", y=["Spread3M", "Spread6M"],
                  title="Monthly spread", grid=True)
plt.axis("tight")
plt.show()

In [ ]:
plt.figure()
(
    all_euribors[["Spread3M", "Spread6M"]]
    .cumsum()
    .join(all_euribors["Date"])
    .plot(x="Date", y=["Spread3M", "Spread6M"], title="Cumulative spread", grid=True)
)
plt.show()

### Interest Paid

In [ ]:
def calculate_payment(N, r, m, n):
    """
        Function calculating the yearly payment of an annuity.

        Parameters:
            N: loan principal
            r: annual interest rate in percentage points
            m: loan marginal in percentage points
            n: number of payments
    """

    interest = (r + m)/100
    return ((1 + interest)**n * interest) / ((1 + interest)**n - 1) * N


def calculate_total_interest(N, margin, interest_rate, loan_length, reset_frequency, payment_start_date, reference_rate):
    """
        Function calculating the total amount interest paid during the lifetime of a loan
        and the realized spread between the interest rate and selected reference rate.

        Parameters:
            N: loan principal
            margin: loan marginal in percentage points
            interest_rate: name of the loan reference rate
            loan_length: length of the loan in years
            reset_frequency: length of the interval between rate resets in months
            payment_start_date: a date in which the loan starts
            reference_rate: name of the selected reference rate
    """

    period_length = reset_frequency / 12 # length of the period for which the interest rate is set in years
    payment_date = payment_start_date
    r = all_euribors[all_euribors["Date"] == payment_date][interest_rate].iloc[0]
    total_interest = []
    realized_spread = []

    while loan_length > 0:
        # calculate yearly payment with the given interest rate
        payment = np.round(calculate_payment(N, r, margin, loan_length), 2)
        if payment < 1:
            break
        # calculate interest paid within the reset period
        interest_paid = N * (r + margin) / 100 * period_length
        total_interest.append(interest_paid) # store the amount of paid interest
        # calculate spread compared to given reference rate
        ref_rate = all_euribors[all_euribors["Date"] == payment_date][reference_rate].iloc[0]
        realized_spread.append(0.0 if r == 0 else (r - ref_rate) * period_length)
        # calculate new loan principal by subtracting total interest from total payment
        N -= payment * period_length - interest_paid
        # roll the loan one period forward
        loan_length -= period_length
        payment_date += pd.DateOffset(months=reset_frequency)
        # set new interest rate
        r = all_euribors[all_euribors["Date"] == payment_date][interest_rate].iloc[0]
        # floor the interest rate to zero so that margin is always paid
        if r < 0:
            r = 0 

    return total_interest, realized_spread


# function to calculate the average cumulative interest paid over calculated interest rate costs
def calculate_average_interest(N, margin, interest_rate, loan_length, reset_frequency, loan_starts, reference_rate="Euribor12M"):
    """
        Function calculating the average cumulative interest paid over calculated interest rate costs
        and average realized spread over the payments.

        Parameters:
            N: loan principal
            margin: loan marginal in percentage points
            interest_rate: name of the loan reference rate
            loan_length: length of the loan in years
            reset_frequency: length of the interval between rate resets in months
            loan_starts: a list of dates in which a loan starts
            reference_rate: name of the selected reference rate
    """
    
    total_interests = []
    realized_spreads = []
    for ls in loan_starts:
        ti, rs = calculate_total_interest(N, margin, interest_rate, loan_length, reset_frequency, ls, reference_rate)
        total_interests.append(ti)
        realized_spreads.append(rs)

    total_interest_paths = (
        pd.DataFrame(total_interests)
        .transpose() # each column is one loan
        .assign(YearNo = np.repeat(range(1, loan_length + 1), 12/reset_frequency))
        .groupby("YearNo")
        .agg(np.sum) # aggregate the interest paid by year
    )

    total_interests = (
        total_interest_paths
        .cumsum() # calculate the cumulative interest paid
        .agg([np.mean, np.std], axis=1) # mean and standard deviation per year
        .rename(lambda x: x.capitalize() + interest_rate, axis=1)
    )

    realized_spreads = (
        pd.DataFrame(realized_spreads)
        .transpose() # each column is one loan
        .assign(YearNo = np.repeat(range(1, loan_length + 1), 12/reset_frequency))
        .groupby("YearNo")
        .agg(np.sum) # aggregate the interest paid by year
        .cumsum() # calculate the cumulative interest paid
        .agg([np.mean, np.std], axis=1) # mean and standard deviation per year
        .rename(lambda x: x.capitalize() + interest_rate, axis=1)
    )

    return total_interests, total_interest_paths, realized_spreads


def calculate_saving_rate(df, base_rate, ref_rate):
    """
        Function calculating the saving rate of cumulative costs.

        Parameters:
            df: a pandas DataFrame containing cumulative costs s.t. total is
                given by the last row
            base_rate: name of the selected interest rate
            ref_rate: name of the selected interest rate
    """

    br = f"Mean{base_rate}"
    rr = f"Mean{ref_rate}"

    return 100 * (df[br].iloc[-1] - df[rr].iloc[-1]) / df[br].iloc[-1]

In [ ]:
# set general parameters for the loan
loan_params = pd.read_csv("./data/loan_parameters.csv")
# generate starting dates for the loan 
loan_starts = [pd.Timestamp("2002-01-01") + pd.DateOffset(months=x) for x in range(12)]

# calculate the average total interest paid and realized spread
# per interest rate given the loan parameters
euribor_12m_ti, euribor_12m_ti_paths, euribor_12m_rs = calculate_average_interest(
    *loan_params[loan_params["interest_rate"] == "Euribor12M"].values.flatten().tolist(),
    loan_starts
)
euribor_6m_ti, euribor_6m_ti_paths, euribor_6m_rs = calculate_average_interest(
    *loan_params[loan_params["interest_rate"] == "Euribor6M"].values.flatten().tolist(),
    loan_starts
)
euribor_3m_ti, euribor_3m_ti_paths, euribor_3m_rs = calculate_average_interest(
    *loan_params[loan_params["interest_rate"] == "Euribor3M"].values.flatten().tolist(),
    loan_starts
)
# join the results into one data frame
total_interests = euribor_12m_ti.join(euribor_6m_ti).join(euribor_3m_ti)
realized_spreads = euribor_12m_rs.join(euribor_6m_rs).join(euribor_3m_rs)

total_interests.tail()

In [ ]:
# calculate the average relative saving in total interest costs over the lifetime of the loan
calculate_saving_rate(total_interests, "Euribor12M", "Euribor6M"), calculate_saving_rate(total_interests, "Euribor12M", "Euribor3M")

In [ ]:
# calculate how many times the interest rate expenses of 12M
# are greater than or equal to another rate on average
(
    euribor_12m_ti_paths
    .cumsum()
    .ge(euribor_6m_ti_paths.cumsum(), axis=1)
    .count(axis=1)
    .agg(np.mean)
)

In [ ]:
(
    euribor_12m_ti_paths
    .cumsum()
    .ge(euribor_3m_ti_paths.cumsum(), axis=1)
    .count(axis=1)
    .agg(np.mean)
)

In [ ]:
# plot the results
fig, ax = plt.subplots()
total_interests.plot.line(y="MeanEuribor12M", yerr="StdEuribor12M", label="Euribor12M", ax=ax, capsize=4, grid=True)
total_interests.plot.line(y="MeanEuribor6M", yerr="StdEuribor6M", label="Euribor6M", ax=ax, capsize=4, grid=True)
total_interests.plot.line(y="MeanEuribor3M", yerr="StdEuribor3M", label="Euribor3M", ax=ax, capsize=4, grid=True)
ax.set_xlabel("Year Index")
ax.set_ylabel("Cumulative Total Interest")
ax.set_title("Loans starting 2002")
plt.show()

fig, ax = plt.subplots()
realized_spreads.plot(y=["MeanEuribor6M", "MeanEuribor3M"], ax=ax, grid=True)
ax.set_xlabel("Year Index")
ax.set_ylabel("Cumulative Realized Spread")
ax.set_title("Loans starting 2002")
plt.show()

In [ ]:
# select the earliest possible starting time
loan_starts_early = [pd.Timestamp("1994-01-01") + pd.DateOffset(months=x) for x in range(12)]

euribor_12m_ti_early, euribor_12m_ti_paths_early, euribor_12m_rs_early = calculate_average_interest(
    *loan_params[loan_params["interest_rate"] == "Euribor12M"].values.flatten().tolist(),
    loan_starts_early
)
euribor_6m_ti_early, euribor_6m_ti_paths_early, euribor_6m_rs_early = calculate_average_interest(
    *loan_params[loan_params["interest_rate"] == "Euribor6M"].values.flatten().tolist(),
    loan_starts_early
)
euribor_3m_ti_early, euribor_3m_ti_paths_early, euribor_3m_rs_early = calculate_average_interest(
    *loan_params[loan_params["interest_rate"] == "Euribor3M"].values.flatten().tolist(),
    loan_starts_early
)
# join the results into one data frame
total_interests_early = euribor_12m_ti_early.join(euribor_6m_ti_early).join(euribor_3m_ti_early)
realized_spreads_early = euribor_12m_rs_early.join(euribor_6m_rs_early).join(euribor_3m_rs_early)

total_interests_early.tail()

In [ ]:
fig, ax = plt.subplots()
total_interests_early.plot.line(y="MeanEuribor12M", yerr="StdEuribor12M", label="Euribor12M", ax=ax, capsize=4, grid=True)
total_interests_early.plot.line(y="MeanEuribor6M", yerr="StdEuribor6M", label="Euribor6M", ax=ax, capsize=4, grid=True)
total_interests_early.plot.line(y="MeanEuribor3M", yerr="StdEuribor3M", label="Euribor3M", ax=ax, capsize=4, grid=True)
ax.set_xlabel("Year Index")
ax.set_ylabel("Cumulative Total Interest")
ax.set_title("Loans starting 1994")
plt.show()

fig, ax = plt.subplots()
realized_spreads_early.plot(y=["MeanEuribor6M", "MeanEuribor3M"], ax=ax, grid=True)
ax.set_xlabel("Year Index")
ax.set_ylabel("Cumulative Realized Spread")
ax.set_title("Loans starting 1994")
plt.show()

In [ ]:
# relative savings in interest expenses
calculate_saving_rate(total_interests_early, "Euribor12M", "Euribor6M"), calculate_saving_rate(total_interests_early, "Euribor12M", "Euribor3M")

In [ ]:
# plot some comparison figures
tmp = ["Euribor12M", "Euribor6M", "Euribor3M"]
plot_y = [f"Mean{x}" for x in tmp]

fig, ax = plt.subplots()
total_interests.plot.line(y=plot_y, label=[f"{x}-02" for x in tmp], ax=ax, grid=True, style="--")
total_interests_early.plot.line(y=plot_y, label=[f"{x}-94" for x in tmp], ax=ax, grid=True)
ax.set_xlabel("Year Index")
ax.set_ylabel("Cumulative Total Interest")
plt.show()

fig, ax = plt.subplots()
realized_spreads.plot(y=plot_y[1:], label=[f"{x}-02" for x in tmp[1:]], ax=ax, grid=True, style="--")
realized_spreads_early.plot(y=plot_y[1:], label=[f"{x}-94" for x in tmp[1:]], ax=ax, grid=True)
ax.set_xlabel("Year Index")
ax.set_ylabel("Cumulative Realized Spread")
plt.show()